# Test Performance

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

import numpy as np
import tensorflow as tf

import fcn16_vgg
import utils
import accuracy

Load dataset. Split dataset into train, validation and test datasets. Define height, width and number of classes.

In [2]:
RESOURCE = '../dataset'
MODEL_PATH = "./models/model-300-5-40.ckpt"

input_set = np.load("input_set.npy")
output_set = np.load("output_set.npy")

train_input_set, train_output_set, test_input_set, test_output_set \
    = utils.train_test_split(input_set, output_set, 0.1)

train_input_set, train_output_set, valid_input_set, valid_output_set \
    = utils.train_test_split(train_input_set, train_output_set, 0.1)
    
height = 180
width = 320
num_classes = 3

Define TensorFlow input and output placeholders.

In [3]:
input_placeholder = tf.placeholder(tf.float32, [None, height, width, num_classes])
output_placeholder = tf.placeholder(tf.float32, [None, height, width, num_classes])

Build FCN-VGG16 neural network.

In [4]:
vgg_fcn = fcn16_vgg.FCN16VGG('./vgg16.npy')

with tf.name_scope("content_vgg"):
    vgg_fcn.build(input_placeholder, train=True, num_classes=num_classes)
    
print('Finished building Network.')

npy file loaded
Layer name: conv1_1
Layer shape: (3, 3, 3, 64)
Layer name: conv1_2
Layer shape: (3, 3, 64, 64)
Layer name: conv2_1
Layer shape: (3, 3, 64, 128)
Layer name: conv2_2
Layer shape: (3, 3, 128, 128)
Layer name: conv3_1
Layer shape: (3, 3, 128, 256)
Layer name: conv3_2
Layer shape: (3, 3, 256, 256)
Layer name: conv3_3
Layer shape: (3, 3, 256, 256)
Layer name: conv4_1
Layer shape: (3, 3, 256, 512)
Layer name: conv4_2
Layer shape: (3, 3, 512, 512)
Layer name: conv4_3
Layer shape: (3, 3, 512, 512)
Layer name: conv5_1
Layer shape: (3, 3, 512, 512)
Layer name: conv5_2
Layer shape: (3, 3, 512, 512)
Layer name: conv5_3
Layer shape: (3, 3, 512, 512)
Layer name: fc6
Layer shape: [7, 7, 512, 4096]
Layer name: fc7
Layer shape: [1, 1, 4096, 4096]
Layer name: fc8
Layer shape: [1, 1, 4096, 1000]
Finished building Network.


In [5]:
def compare(predicted_data, real_data):
    """Compare predicted image with real image.

    Args:
        predicted_data: numpy array, int32 - [height, width].
            Array of the prediction.
        real_data: numpy array, int32 - [height, width].
            Array of the real.

    Returns:
        result: float32.
            Similarity of the images.
    """
    num_equals = 0
    height, width = predicted_data.shape[:2]

    match_pixels = (predicted_data == real_data).flatten()
    result = match_pixels[match_pixels == True]
    result = 100.0 * result.shape[0] / (height * width)
    return result

Load TensorFlow trained model. Measure accuracy and speed.

In [6]:
# Initializing the variables.
init = tf.global_variables_initializer()

# Saver op to save and restore all the variables
saver = tf.train.Saver()

# With CPU mini-batch size can be bigger.

average_time = 0.0
average_accuracy = 0.0
with tf.device('/cpu:0'):
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allow_growth = True

    with tf.Session(config=config) as sess:
        # Restore model weights from previously saved model.
        saver.restore(sess, MODEL_PATH)
        
        for i in range(test_input_set.shape[0]):
            start_time = int(round(time.time() * 1000))
            
            prediction = sess.run(vgg_fcn.pred_up, feed_dict={input_placeholder: [test_input_set[i]]})
            
            # Time measurement.
            end_time = int(round(time.time() * 1000))
            average_time += end_time - start_time
            print(end_time - start_time)
            average_accuracy += accuracy.compare(prediction[0], test_output_set[i].argmax(2))
            print(accuracy.compare(prediction[0], test_output_set[i].argmax(2)))

average_time /= test_input_set.shape[0]
print("Average time: " + str(average_time))

average_accuracy /= test_input_set.shape[0]
print("Average accuracy: " + str(average_accuracy))


972
93.12326388888889
109
99.11979166666667
94
94.953125
110
98.0625
109
97.65798611111111
108
96.58506944444444
104
99.35069444444444
105
97.25694444444444
100
95.62673611111111
102
95.41840277777777
101
98.08333333333333
103
99.015625
102
92.92361111111111
103
98.97743055555556
98
99.45486111111111
105
91.25868055555556
97
93.49652777777777
109
95.92361111111111
94
98.80208333333333
110
97.86458333333333
109
92.3125
94
95.95486111111111
109
98.73090277777777
94
91.9375
109
90.52430555555556
110
96.01388888888889
93
95.01909722222223
105
98.56944444444444
100
97.46701388888889
103
94.36111111111111
Average time: 132.03333333333333
Average accuracy: 96.12818287037037
